In [ ]:
import os
import torch
from transformers import pipeline

# ==========================================
# KONFIGURASI LOKASI MODEL (WAJIB DIGANTI)
# ==========================================
# Masukkan path folder hasil ekstrak ZIP masing-masing model
# Gunakan tanda miring biasa (/) agar aman di Windows

# 1. Lokasi Model Indo -> Madura
PATH_INDO_MAD = "D:/UNAIR/NLP/Project_Madura/model_indo_ke_madura"

# 2. Lokasi Model Madura -> Indo
PATH_MAD_INDO = "D:/UNAIR/NLP/Project_Madura/model_madura_ke_indo"

# ==========================================

def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

def muat_model(path_model):
    """Fungsi untuk memuat model ke memori (GPU/CPU)"""
    print(f"\n⏳ Sedang memuat model dari: {path_model}...")
    try:
        # device=0 artinya pakai GPU RTX 3050. Kalau error, ganti device=-1 (CPU)
        device_id = 0 if torch.cuda.is_available() else -1
        penerjemah = pipeline("translation", model=path_model, tokenizer=path_model, device=device_id)
        print("✅ Model Berhasil Dimuat!")
        return penerjemah
    except Exception as e:
        print(f"❌ Gagal memuat model. Error:\n{e}")
        return None

def main():
    while True:
        clear_screen()
        print("="*40)
        print("   🇮🇩  APLIKASI PENERJEMAH MADURA  🐂   ")
        print("="*40)
        print("Pilih Mode Penerjemahan:")
        print("1. Indonesia  ➡️  Madura")
        print("2. Madura     ➡️  Indonesia")
        print("0. Keluar Aplikasi")
        
        pilihan = input("\nMasukkan angka (0-2): ")

        if pilihan == '0':
            print("Sampai jumpa! 👋")
            break

        # Tentukan Model dan Prefix berdasarkan pilihan
        if pilihan == '1':
            path_aktif = PATH_INDO_MAD
            # Prefix harus SAMA PERSIS dengan saat training
            prefix = "terjemahkan dari Bahasa Indonesia ke Bahasa Madura: "
            judul = "🇮🇩 INDONESIA -> MADURA 🐂"
        elif pilihan == '2':
            path_aktif = PATH_MAD_INDO
            # Prefix harus SAMA PERSIS dengan saat training
            prefix = "terjemahkan dari Bahasa Madura ke Bahasa Indonesia: "
            judul = "🐂 MADURA -> INDONESIA 🇮🇩"
        else:
            print("Pilihan tidak valid.")
            input("Tekan Enter untuk ulang...")
            continue

        # Cek apakah folder ada
        if not os.path.exists(path_aktif):
            print(f"\n❌ Folder model tidak ditemukan di:\n{path_aktif}")
            print("Mohon cek baris konfigurasi di dalam kode.")
            input("Tekan Enter untuk kembali...")
            continue

        # Load Model
        model_sekarang = muat_model(path_aktif)
        if not model_sekarang:
            input("Tekan Enter untuk kembali...")
            continue

        # Loop Interaksi Chat
        while True:
            print(f"\n--- MODE: {judul} ---")
            text = input("Ketik kalimat (ketik 'q' untuk ganti mode): ")
            
            if text.lower() == 'q':
                # Hapus model dari memori GPU agar tidak penuh saat ganti mode
                del model_sekarang
                torch.cuda.empty_cache()
                break
            
            # Proses Penerjemahan
            # max_length sesuaikan dengan training (100 atau 128)
            try:
                full_input = prefix + text
                hasil = model_sekarang(full_input, max_length=128)
                terjemahan = hasil[0]['translation_text']
                
                print(f"✅ Hasil: {terjemahan}")
                print("-" * 30)
            except Exception as e:
                print(f"Error saat menerjemahkan: {e}")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nAplikasi ditutup.")